In [ ]:
from pathlib import Path

model_store_path = Path(".").resolve().parent / "AutogluonModels"
model_store_path

In [ ]:
# alias the predictor.path attribute, which points to the model store: this is the input
model_paths: list[Path] = list(model_store_path.glob("**/ag-*"))
model_paths

In [ ]:
predictor_path: str = str(model_paths[-1])

possible_log_file_paths: list[Path] = list(
    Path(predictor_path).glob("**/predictor_log.txt")
)
if not possible_log_file_paths:
    raise FileNotFoundError(f"No log file found in model path {predictor_path}")

log_file_path: Path = possible_log_file_paths[0]
log_file_path

In [ ]:
import ast

log_data: str = log_file_path.read_text()
log_lines: list[str] = log_data.splitlines()


log_dicts: list[dict] = [
    ast.literal_eval(line)
    for line in log_lines
    if line.startswith("{") and line.endswith("}")
]
log_dicts[:3]  # show first 3 entries

In [ ]:
# now plot the loss curve and learning rate in one plot with learning rate on secondary y-axis
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))
losses = [d["loss"] for d in log_dicts if "loss" in d]
lrs = [d["learning_rate"] for d in log_dicts if "learning_rate" in d]

ax1.plot(losses, color="blue", label="Train Loss")
ax1.set_xlabel("Iteration")
ax1.set_xticks(
    ticks=range(0, len(losses) + 1, len(losses) // 10),
    labels=range(0, len(losses) * 100 + 1, len(losses) * 10),  # type: ignore
)
ax1.set_ylabel("Train Loss", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")
ax2 = ax1.twinx()
ax2.plot(lrs, color="red", label="Learning Rate")
ax2.set_ylabel("Learning Rate", color="red")
ax2.tick_params(axis="y", labelcolor="red")
fig.suptitle("Loss Curve and Learning Rate")
plt.show()